In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
starwis = pd.read_csv("../original_data/starwis_satamakatu.csv")
weather = pd.read_csv("../original_data/Tie_9_Jyväskylä.csv")

In [3]:
print(starwis.columns)

Index(['ts', 'Friction', 'Road_Surface_Temperature', 'Water_Film_Surface_mm'], dtype='object')


In [4]:
print(weather.columns)

Index(['ts', 'airTemp', 'roadTemp', 'friction3', 'humidity', 'dewpoint',
       'precipitation'],
      dtype='object')


Combine starwis and weather, since they share same time period with sameish 10minute step

In [5]:
# drop from weather station dataset:
# friction3 - there are 100k missing values
# roadTemp - we already have it in more accurate form in starwis dataset  

weather.drop(["friction3", "roadTemp"], axis=1, inplace=True)


In [6]:
# Convert the 'timestamp' columns in both dataframes to datetime format
weather['ts'] = pd.to_datetime(weather['ts'])
starwis['ts'] = pd.to_datetime(starwis['ts'], format='mixed')

# Sort the dataframes by 'timestamp' before using merge_asof
weather = weather.sort_values('ts')
starwis = starwis.sort_values('ts')

In [7]:
# Perform an asof merge, joining on the nearest timestamp within a tolerance
# Set the tolerance to a suitable time difference (e.g., 5 minutes)
# (if time difference is more than 5 minutes, data points won't be joined)
df = pd.merge_asof(weather, starwis, on='ts', direction='nearest', tolerance=pd.Timedelta('5min'))

In [8]:
# Drop data points that couldn't be joined and other missing values
df.dropna(inplace=True)

In [9]:
df["ts"] = pd.to_datetime(df.ts)
df.set_index("ts", inplace=True)

In [10]:
# Filter data between november and march
df_2021 = df[(df.index >= "2021-11-01 00:00:00") & (df.index < "2022-05-01 00:00:00")]
df_2022 = df[(df.index >= "2022-11-01 00:00:00") & (df.index < "2023-05-01 00:00:00")]
df_2023 = df[(df.index >= "2023-11-01 00:00:00") & (df.index < "2024-05-01 00:00:00")]

In [11]:
df = pd.concat([df_2021, df_2022, df_2023])

In [12]:
# save merged dataset to .csv file
df.to_csv("../ready_data/merged_starwis_and_weather.csv")